Loading the dataset csv's into data frames and printing the first 10 values in the data set for each data frame to ensure that the data is being read and stored correctly

In [ ]:
import pandas as pd

dataset_1 = pd.read_csv("CO2 emission by countries.csv")
dataset_2 = pd.read_csv("Historical Tropical Storm.csv")
dataset_3 = pd.read_csv("global_temps.csv")

print("From Dataset 1: CO2 emission by countries")
print(dataset_1.head(10))
print("\n")

print("From Dataset 2: Historical Tropical Storm")
print(dataset_2.head(10))
print("\n")

print("From Dataset 3: global_temps")
print(dataset_3.head(10))
print("\n")